In [ ]:
import pandas as pd
from helper import load_config, consolidate_financials, highlight_ratio

config = load_config()
symbols, functions, period = config['symbols'], config['functions'], config['period']

consolidate_financials(symbols=symbols, functions=functions, period=period)

data_dir = '../../data'

for symbol in symbols:
    file_path = f'{data_dir}/consolidated/{symbol}.json'

    # Convert the dictionary to a DataFrame
    df = pd.read_json(file_path)

    # Single dictionary containing both ratio data and threshold comparison
    ratios = {
        'Gross Margin': {
            'ratio': df.loc['grossProfit'] / df.loc['totalRevenue'],
            'threshold': 0.4,
            'comparison': 'greater'
        },
        'SG&A Margin': {
            'ratio': df.loc['sellingGeneralAndAdministrative'] / df.loc['totalRevenue'],
            'threshold': 0.3,
            'comparison': 'less'
        },
        'R&D Margin': {
            'ratio': df.loc['researchAndDevelopment'] / df.loc['grossProfit'],
            'threshold': 0.3,
            'comparison': 'less'
        },
        'Interest Expense Margin': {
            'ratio': df.loc['interestExpense'] / df.loc['operatingIncome'],
            'threshold': 0.15,
            'comparison': 'less'
        },
        'Income Tax Margin': {
            'ratio': df.loc['incomeTaxExpense'] / df.loc['incomeBeforeTax'],
            'threshold': 0.2,
            'comparison': ''
        },
        'Profit Margin': {
            'ratio': df.loc['netIncome'] / df.loc['totalRevenue'],
            'threshold': 0.2,
            'comparison': 'greater'
        },
        'Earnings Per Share': {
            'ratio': df.loc['reportedEPS'],
            'threshold': 0.2,
            'comparison': ''
        },
        'Adjusted Debt to Equity': {
            'ratio': df.loc['totalLiabilities'] / (df.loc['totalShareholderEquity'] + df.loc['treasuryStock']),
            'threshold': 0.8,
            'comparison': 'less'
        },
    }

    # Create the combined DataFrame
    combined_data = {}
    
    # Loop over the ratios dictionary
    for name, ratio in ratios.items():
        # Store each ratio's data in the dictionary with the ratio name as the key
        combined_data[name] = ratio['ratio']
    
    # Convert the dictionary to a DataFrame with dates as the columns and names as the index
    df_combined = pd.DataFrame(combined_data)
    
    # Transpose to get the desired format (names as rows, dates as columns)
    df_combined = df_combined.T
    
    # Convert the columns to datetime (if not already)
    df_combined.columns = pd.to_datetime(df_combined.columns, errors='coerce').to_period(period)
    if period == "Y":
        # Group by year and take the maximum value for each year (latest date for each year)
        df_combined = (df_combined.T
                       .groupby(df_combined.columns.year)
                       .max()
                       .T)
    
    # Sort the columns by year in descending order
    df_combined = df_combined[sorted(df_combined.columns, reverse=True)]
    
    # Drop columns with too many NaN values (optional threshold, for example, 50% NaNs)
    df_filtered_clean = df_combined.dropna(axis=1, thresh=df_combined.shape[0] * 0.5)

    # Apply the highlight function to the DataFrame
    styled_df = df_filtered_clean.style.apply(highlight_ratio, margins=ratios, axis=1)
    
    # Display the styled DataFrame in Jupyter notebooks
    print(f"Displaying data for: {symbol}")
    
    display(styled_df)
    
    html = styled_df.to_html()  # Render the styled DataFrame to HTML
    
    # Save the HTML to a file
    with open(f"{data_dir}/report/{symbol}-{period}.html", "w") as file:
        file.write(html)